# Librosa API
#### Objective
4. Build librosa class
    * Convert each preview in /data/audio to spectogram
    * Add spectograms to subfolder /data/spectograms
    * Create final dataframe (like with audio file conversion) that contains spectrogram paths
    

In [25]:
import sys
sys.path.append('../')
import os
import matplotlib.pyplot as plt

#for loading and visualizing audio files
import librosa
import librosa.display
import soxr

#To load csvs and adjust arrays
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import string

In [2]:
data = pd.read_csv('../data/tracklist_spectrograms.csv', sep=",")
data.head()

,track_name,artist,album,track_url,preview_url,artist_url,genres,file_location,spectrogram
0,Guillotine,Stray From The Path,Guillotine,https://api.spotify.com/v1/tracks/5mWs0zabZrRE...,https://p.scdn.co/mp3-preview/dbd5861d09ee540c...,https://api.spotify.com/v1/artists/5tGG1slV9pk...,"['deathcore', 'punk', 'melodic', 'hardcore', '...",../data/audio/Guillotine-Stray From The Path.wav,../data/images/spectrograms/Guillotine_Stray F...
1,Vermin,Allegaeon,DAMNUM,https://api.spotify.com/v1/tracks/6FDLIsQnjDx5...,https://p.scdn.co/mp3-preview/7d8fba2941244a72...,https://api.spotify.com/v1/artists/5NIvyOvy39l...,"['melodeath', 'metal', 'melodic', 'death', 'te...",../data/audio/Vermin-Allegaeon.wav,../data/images/spectrograms/Vermin_Allegaeon_s...
2,Put Your Back Into The Oar,Amon Amarth,Put Your Back Into The Oar,https://api.spotify.com/v1/tracks/4fY2JR0REUiy...,https://p.scdn.co/mp3-preview/d2253d713457b378...,https://api.spotify.com/v1/artists/3pulcT2wt7F...,"['death', 'metal', 'melodic', 'melodeath', 'fo...",../data/audio/Put Your Back Into The Oar-Amon ...,../data/images/spectrograms/Put Your Back Into...
3,Make Believe,Memphis May Fire,Make Believe,https://api.spotify.com/v1/tracks/5NLi9YVXt9pM...,https://p.scdn.co/mp3-preview/50d174c4df0bdff7...,https://api.spotify.com/v1/artists/7cNNNhdJDrt...,"['melodic', 'metalcore', 'post-hardcore', 'scr...",../data/audio/Make Believe-Memphis May Fire.wav,../data/images/spectrograms/Make Believe_Memph...
4,Dystopia (feat. Christoph Wieczorek of Annisokay),Caliban,Dystopia (feat. Christoph Wieczorek of Annisokay),https://api.spotify.com/v1/tracks/2WaaniEgnI8p...,https://p.scdn.co/mp3-preview/927ee1c9965680c1...,https://api.spotify.com/v1/artists/1oeo6cC9Fqa...,"['deathcore', 'metalcore', 'post-hardcore', 'm...",../data/audio/Dystopia feat Christoph Wieczore...,../data/images/spectrograms/Dystopia feat Chri...


In [3]:
data.file_location.duplicated().sum()

0

In [4]:
data.shape

(688, 9)

In [5]:
data['spectrogram'] = ''

In [6]:
data.head()

,track_name,artist,album,track_url,preview_url,artist_url,genres,file_location,spectrogram
0,Guillotine,Stray From The Path,Guillotine,https://api.spotify.com/v1/tracks/5mWs0zabZrRE...,https://p.scdn.co/mp3-preview/dbd5861d09ee540c...,https://api.spotify.com/v1/artists/5tGG1slV9pk...,"['deathcore', 'punk', 'melodic', 'hardcore', '...",../data/audio/Guillotine-Stray From The Path.wav,
1,Vermin,Allegaeon,DAMNUM,https://api.spotify.com/v1/tracks/6FDLIsQnjDx5...,https://p.scdn.co/mp3-preview/7d8fba2941244a72...,https://api.spotify.com/v1/artists/5NIvyOvy39l...,"['melodeath', 'metal', 'melodic', 'death', 'te...",../data/audio/Vermin-Allegaeon.wav,
2,Put Your Back Into The Oar,Amon Amarth,Put Your Back Into The Oar,https://api.spotify.com/v1/tracks/4fY2JR0REUiy...,https://p.scdn.co/mp3-preview/d2253d713457b378...,https://api.spotify.com/v1/artists/3pulcT2wt7F...,"['death', 'metal', 'melodic', 'melodeath', 'fo...",../data/audio/Put Your Back Into The Oar-Amon ...,
3,Make Believe,Memphis May Fire,Make Believe,https://api.spotify.com/v1/tracks/5NLi9YVXt9pM...,https://p.scdn.co/mp3-preview/50d174c4df0bdff7...,https://api.spotify.com/v1/artists/7cNNNhdJDrt...,"['melodic', 'metalcore', 'post-hardcore', 'scr...",../data/audio/Make Believe-Memphis May Fire.wav,
4,Dystopia (feat. Christoph Wieczorek of Annisokay),Caliban,Dystopia (feat. Christoph Wieczorek of Annisokay),https://api.spotify.com/v1/tracks/2WaaniEgnI8p...,https://p.scdn.co/mp3-preview/927ee1c9965680c1...,https://api.spotify.com/v1/artists/1oeo6cC9Fqa...,"['deathcore', 'metalcore', 'post-hardcore', 'm...",../data/audio/Dystopia feat Christoph Wieczore...,


In [7]:
data.reset_index(drop=True, inplace=True)

In [30]:
def make_spectrograms(df):
    img_path = "../data/images/spectrograms/"
    punct = string.punctuation
    
    for row in tqdm(range(len(df))):
#         audio_path = df['file_location'][row]
        track = re.sub(r"[^a-zA-Z0-9]", "", df['track_name'][row])
        artist = re.sub(r"[^a-zA-Z0-9]", "", df['artist'][row])
        spect = f"{track}_{artist}_spectro.png"

        path = df['file_location'][row]
        
        x, sr = librosa.core.load(path, sr=22050, mono=True, offset=0.0, duration=29, res_type='soxr_vhq')
        
        # Convert waveform to spectrogram
        X = librosa.stft(x)
        Xdb = librosa.amplitude_to_db(abs(X))
        plt.figure(figsize=(14, 5))
        librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='log')
        plt.savefig(img_path+spect)
        plt.close()
        data.at[row, 'spectrogram'] = img_path+spect

# RUN ONLY ONCE

In [32]:
make_spectrograms(data)

100%|████████████████████████████████████████████████████████████████████████████████| 688/688 [12:24<00:00,  1.08s/it]


In [29]:
data.head()

,track_name,artist,album,track_url,preview_url,artist_url,genres,file_location,spectrogram
0,Guillotine,Stray From The Path,Guillotine,https://api.spotify.com/v1/tracks/5mWs0zabZrRE...,https://p.scdn.co/mp3-preview/dbd5861d09ee540c...,https://api.spotify.com/v1/artists/5tGG1slV9pk...,"['deathcore', 'punk', 'melodic', 'hardcore', '...",../data/audio/Guillotine-Stray From The Path.wav,../data/images/spectrograms/Guillotine_StrayFr...
1,Vermin,Allegaeon,DAMNUM,https://api.spotify.com/v1/tracks/6FDLIsQnjDx5...,https://p.scdn.co/mp3-preview/7d8fba2941244a72...,https://api.spotify.com/v1/artists/5NIvyOvy39l...,"['melodeath', 'metal', 'melodic', 'death', 'te...",../data/audio/Vermin-Allegaeon.wav,../data/images/spectrograms/Vermin_Allegaeon_s...
2,Put Your Back Into The Oar,Amon Amarth,Put Your Back Into The Oar,https://api.spotify.com/v1/tracks/4fY2JR0REUiy...,https://p.scdn.co/mp3-preview/d2253d713457b378...,https://api.spotify.com/v1/artists/3pulcT2wt7F...,"['death', 'metal', 'melodic', 'melodeath', 'fo...",../data/audio/Put Your Back Into The Oar-Amon ...,../data/images/spectrograms/PutYourBackIntoThe...
3,Make Believe,Memphis May Fire,Make Believe,https://api.spotify.com/v1/tracks/5NLi9YVXt9pM...,https://p.scdn.co/mp3-preview/50d174c4df0bdff7...,https://api.spotify.com/v1/artists/7cNNNhdJDrt...,"['melodic', 'metalcore', 'post-hardcore', 'scr...",../data/audio/Make Believe-Memphis May Fire.wav,../data/images/spectrograms/MakeBelieve_Memphi...
4,Dystopia (feat. Christoph Wieczorek of Annisokay),Caliban,Dystopia (feat. Christoph Wieczorek of Annisokay),https://api.spotify.com/v1/tracks/2WaaniEgnI8p...,https://p.scdn.co/mp3-preview/927ee1c9965680c1...,https://api.spotify.com/v1/artists/1oeo6cC9Fqa...,"['deathcore', 'metalcore', 'post-hardcore', 'm...",../data/audio/Dystopia feat Christoph Wieczore...,../data/images/spectrograms/DystopiafeatChrist...


In [33]:
data.to_csv("../data/tracklist_spectrograms.csv", sep=",", index=False)